In [1]:
import tensorflow as tf
from data_manipulation.data import Data
from keras.applications import inception_v3
from keras.models import Model
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
import keras.optimizers as optimizers

num_classes = 4
inception_v3_no_top = inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(224,224,3))
net = GlobalAveragePooling2D()(inception_v3_no_top.output)
logits = Dense(num_classes, activation=None)(net)
inception_v3_model = Model(inputs=inception_v3_no_top.input, outputs=logits)
inception_v3_model.summary()

Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [2]:
# Optimizer and Loss function.
opt = optimizers.Adam(lr=0.002)
inception_v3_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [3]:
height = 224
width = height
dataset_path = '/Users/adalbertoclaudioquiros/Documents/Code/UofG/PhD/Cancer_TMA_Generative'
dataset = 'NKI'
marker = 'HE'

epochs = 40
batch_size = 128

data = Data(dataset='nki', marker='he', patch_h=height, patch_w=width, n_channels=3, batch_size=batch_size, project_path=dataset_path)

In [5]:
with tf.Graph().as_default():
    input_shape = data.training.shape[1:]
    input_img = tf.placeholder(tf.float32, (None, height,width,3), name='image')
    inception_v3_no_top = inception_v3.InceptionV3(include_top=False, weights='imagenet', input_tensor=input_img, input_shape=input_shape)
    net = GlobalAveragePooling2D()(inception_v3_no_top.output)
    logits = Dense(num_classes, activation=None)(net)
    inception_v3_model = Model(inputs=inception_v3_no_top.input, outputs=logits)
    
    # Optimizer and Loss function.
    opt = optimizers.Adam(lr=0.002)
    inception_v3_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    with tf.Session() as sess:
        sess.run(tf.global_variables_intializer())
        for epoch in range(1, epochs+1):
            for batch_images, batch_labels in data.training:
                output_val = sess.run(logits, {input_img:batch_images})

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder:0", shape=(3, 3, 3, 32), dtype=float32) is not an element of this graph.